In [ ]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

In [ ]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float16", 'snap_TX': 'float16', 'snap_WI': 'float16' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float16" }

In [ ]:
pd.options.display.max_columns = 50

In [ ]:
h = 28 
max_lags = 57
tr_last = 1913
fday = datetime(2016,4, 25) 
fday

datetime.datetime(2016, 4, 25, 0, 0)

In [ ]:
def create_dt(is_train = True, nrows = None, first_day = 1200):
    prices = pd.read_csv("sell_prices.csv", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
            
    cal = pd.read_csv("calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
    
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float16" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    dt = pd.read_csv("sales_train_validation.csv", 
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()
    
    if not is_train:
        for day in range(tr_last+1, tr_last+ 28 +1):
            dt[f"d_{day}"] = np.nan
    
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    
    dt = dt.merge(cal, on= "d", copy = False)
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    return dt

In [ ]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    
    
    date_features = {
        
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [ ]:
FIRST_DAY = 1250 # If you want to load all the data set it to '1' -->  Great  memory overflow  risk !

In [ ]:
%%time

df = create_dt(is_train=True, first_day= FIRST_DAY)
df.shape

CPU times: user 15 s, sys: 143 ms, total: 15.2 s
Wall time: 15.2 s


In [ ]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1250,1.0,2014-07-01,11422,5,4,7,2014,0,0,0,0,1.0,1.0,0.0,8.257812
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1251,0.0,2014-07-02,11422,6,5,7,2014,0,0,0,0,1.0,0.0,1.0,8.257812
2,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1252,0.0,2014-07-03,11422,4,6,7,2014,0,0,0,0,1.0,1.0,1.0,8.257812
3,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1253,2.0,2014-07-04,11422,0,7,7,2014,10,2,0,0,1.0,0.0,0.0,8.257812
4,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_1250,1.0,2014-07-01,11422,5,4,7,2014,0,0,0,0,1.0,1.0,0.0,3.970703


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19882196 entries, 0 to 19882195
Data columns (total 22 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float16       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       float16       
 19  snap_TX       float16       
 20  snap_WI       float16       
 21  sell_price    float16       
dtypes: datetime64[ns](1), float16(5), int16(14), object(2)
memory us

In [ ]:
%%time

create_fea(df)
df.shape

CPU times: user 2min 4s, sys: 537 ms, total: 2min 5s
Wall time: 2min 5s


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19882196 entries, 0 to 19882195
Data columns (total 31 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float16       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       float16       
 19  snap_TX       float16       
 20  snap_WI       float16       
 21  sell_price    float16       
 22  lag_7         float16       
 23  lag_28        float16       
 

In [ ]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1250,1.0,2014-07-01,11422,5,4,7,2014,0,0,0,0,1.0,1.0,0.0,8.257812,NaN,NaN,NaN,NaN,NaN,NaN,27,3,1
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1251,0.0,2014-07-02,11422,6,5,7,2014,0,0,0,0,1.0,0.0,1.0,8.257812,NaN,NaN,NaN,NaN,NaN,NaN,27,3,2
2,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1252,0.0,2014-07-03,11422,4,6,7,2014,0,0,0,0,1.0,1.0,1.0,8.257812,NaN,NaN,NaN,NaN,NaN,NaN,27,3,3
3,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1253,2.0,2014-07-04,11422,0,7,7,2014,10,2,0,0,1.0,0.0,0.0,8.257812,NaN,NaN,NaN,NaN,NaN,NaN,27,3,4
4,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_1250,1.0,2014-07-01,11422,5,4,7,2014,0,0,0,0,1.0,1.0,0.0,3.970703,NaN,NaN,NaN,NaN,NaN,NaN,27,3,1


In [ ]:
df.dropna(inplace = True)
df.shape

(18205246, 31)

In [ ]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]

In [ ]:
# train_data = lgb.Dataset(X_train, label = y_train, categorical_feature=cat_feats, free_raw_data=False)
# fake_valid_inds = np.random.choice(len(X_train), 1000000, replace = False)
# fake_valid_data = lgb.Dataset(X_train.iloc[fake_valid_inds], label = y_train.iloc[fake_valid_inds],categorical_feature=cat_feats,
#                              free_raw_data=False)   # This is just a subsample of the training set, not a real validation set !

In [ ]:
%%time

np.random.seed(777)

fake_valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds], 
                         categorical_feature=cat_feats, free_raw_data=False)
fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                              categorical_feature=cat_feats,
                 free_raw_data=False)# This is a random sample, we're not gonna apply any time series train-test-split tricks here!

CPU times: user 7.84 s, sys: 234 ms, total: 8.07 s
Wall time: 8.08 s


In [ ]:
del df, X_train, y_train, fake_valid_inds,train_inds ; gc.collect()

43

In [ ]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 1200,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
}

In [ ]:
%%time

m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=20) 

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[20]	valid_0's rmse: 2.47913
[40]	valid_0's rmse: 2.20436
[60]	valid_0's rmse: 2.13208
[80]	valid_0's rmse: 2.1097
[100]	valid_0's rmse: 2.09861
[120]	valid_0's rmse: 2.09174
[140]	valid_0's rmse: 2.08561
[160]	valid_0's rmse: 2.07925
[180]	valid_0's rmse: 2.07419
[200]	valid_0's rmse: 2.06874
[220]	valid_0's rmse: 2.06526
[240]	valid_0's rmse: 2.06189
[260]	valid_0's rmse: 2.05918
[280]	valid_0's rmse: 2.05495
[300]	valid_0's rmse: 2.05222
[320]	valid_0's rmse: 2.04927
[340]	valid_0's rmse: 2.04627
[360]	valid_0's rmse: 2.04466
[380]	valid_0's rmse: 2.04154
[400]	valid_0's rmse: 2.03946
[420]	valid_0's rmse: 2.03659
[440]	valid_0's rmse: 2.03384
[460]	valid_0's rmse: 2.03226
[480]	valid_0's rmse: 2.02954
[500]	valid_0's rmse: 2.02719
[520]	valid_0's rmse: 2.02568
[540]	valid_0's rmse: 2.02331
[560]	valid_0's rmse: 2.02208
[580]	valid_0's rmse: 2.02039
[600]	valid_0's rmse: 2.01822
[620]	valid_0's rmse: 2.01711
[640]	valid_0's rmse: 2.01585
[660]	valid_0's rmse: 2.01374
[680]	valid_0's

In [ ]:
m_lgb.save_model("model.lgb")

In [ ]:
%%time

alphas = [1.028, 1.023, 1.018]
weights = [1/len(alphas)]*len(alphas)
sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):

    te = create_dt(False)
    cols = [f"F{i}" for i in range(1,29)]

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)
        print(tdelta, day)
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]
        te.loc[te.date == day, "sales"] = alpha*m_lgb.predict(tst) # magic multiplier by kyakovlev



    te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
#     te_sub.loc[te.date >= fday+ timedelta(days=h), "id"] = te_sub.loc[te.date >= fday+timedelta(days=h), 
#                                                                           "id"].str.replace("validation$", "evaluation")
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0., inplace = True)
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    te_sub.to_csv(f"submission_{icount}.csv",index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)


sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission.csv",index=False)

0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2016-05-10 00:00:00
16 2016-05-11 00:00:00
17 2016-05-12 00:00:00
18 2016-05-13 00:00:00
19 2016-05-14 00:00:00
20 2016-05-15 00:00:00
21 2016-05-16 00:00:00
22 2016-05-17 00:00:00
23 2016-05-18 00:00:00
24 2016-05-19 00:00:00
25 2016-05-20 00:00:00
26 2016-05-21 00:00:00
27 2016-05-22 00:00:00
0 1.028 0.3333333333333333
0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2

In [ ]:
sub.head(10)

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.814120,0.814102,0.819704,0.790868,1.070607,1.274954,1.109466,1.047166,0.874637,0.875248,0.906394,1.007390,1.369780,1.203721,0.972184,0.831774,0.845550,0.830740,0.924883,1.475741,1.179359,0.924693,0.775077,0.753607,0.753322,0.884022,1.246887,1.194889
1,FOODS_1_001_CA_2_validation,0.827767,0.851664,0.802941,1.000511,1.128737,1.302818,0.959993,0.938261,0.899293,0.938916,0.896932,1.079171,1.477458,1.251703,0.963205,0.926881,0.977913,0.915126,1.048127,1.524011,1.225024,0.785503,0.724368,0.743205,0.683557,0.899753,1.366119,1.386338
2,FOODS_1_001_CA_3_validation,0.980625,0.935927,0.909070,0.912756,0.974237,1.284396,1.374333,1.130768,1.094884,0.987600,0.914143,1.064914,1.642326,1.321327,1.111399,1.048785,0.940430,0.953903,1.050313,1.864014,1.559425,0.991076,0.936922,0.891266,0.873471,0.945846,1.520878,1.492993
3,FOODS_1_001_CA_4_validation,0.407572,0.377605,0.405289,0.408070,0.440113,0.492494,0.472862,0.426720,0.428929,0.437929,0.472682,0.416755,0.521484,0.340111,0.381661,0.373534,0.406696,0.414665,0.394739,0.574622,0.482943,0.390142,0.379798,0.382634,0.388205,0.408942,0.568068,0.465343
4,FOODS_1_001_TX_1_validation,0.216080,0.213688,0.196800,0.198540,0.233377,0.237391,0.233892,0.562616,0.524269,0.420445,0.435206,0.464022,0.521812,0.372432,0.408809,0.370878,0.395739,0.428635,0.414615,0.533937,0.368360,0.276809,0.252867,0.245947,0.245058,0.228688,0.339907,0.314175
5,FOODS_1_001_TX_2_validation,0.470733,0.460340,0.476961,0.473457,0.482195,0.595634,0.556859,0.510213,0.495729,0.478012,0.510077,0.562489,0.751595,0.511397,0.515045,0.485979,0.487277,0.482460,0.551406,0.731293,0.654263,0.454769,0.433167,0.427871,0.426968,0.482951,0.633648,0.578925
6,FOODS_1_001_TX_3_validation,0.402604,0.371384,0.384879,0.458100,0.466712,0.528676,0.523461,0.480735,0.507992,0.483889,0.474660,0.561498,0.659268,0.430342,0.483718,0.445683,0.472047,0.477295,0.506449,0.650575,0.575131,0.428617,0.411295,0.411072,0.447232,0.454772,0.612935,0.524007
7,FOODS_1_001_WI_1_validation,0.340076,0.407987,0.394239,0.394481,0.475236,0.541119,0.617818,0.490059,0.480508,0.481185,0.608129,0.603631,0.855090,0.531767,0.533247,0.536711,0.559881,0.563579,0.662321,0.966922,0.812008,0.540324,0.558261,0.549477,0.553294,0.674459,1.020956,0.866780
8,FOODS_1_001_WI_2_validation,0.294487,0.308947,0.329467,0.321181,0.381392,0.387950,0.379639,0.408653,0.463029,0.410382,0.417767,0.486473,0.521690,0.357373,0.482327,0.364261,0.483553,0.434041,0.414743,0.549787,0.503410,0.337071,0.332608,0.342320,0.337356,0.363287,0.434471,0.399931
9,FOODS_1_001_WI_3_validation,0.251271,0.240971,0.236531,0.242274,0.281156,0.342563,0.318538,0.326756,0.325019,0.364153,0.368812,0.428896,0.505470,0.348528,0.327153,0.319128,0.325589,0.326082,0.359177,0.511533,0.468096,0.308294,0.298544,0.309644,0.307982,0.361403,0.525919,0.396414


In [ ]:
sub.id.nunique(), sub["id"].str.contains("validation$").sum()

(60980, 30490)

In [ ]:
sub.shape

(60980, 29)